In [1]:
import os

from urllib.request import urlretrieve
from getpass import getpass

from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI

In [2]:
url = 'https://raw.githubusercontent.com/RyoWakabayashi/elixir-learning/main/livebooks/bumblebee/colab/momotaro.txt'
filename = 'momotaro.txt'

urlretrieve(url, filename)

('momotaro.txt', <http.client.HTTPMessage at 0x10f799190>)

In [3]:
loader = TextLoader(filename)
document = loader.load()

In [4]:
def split_text(docs: list) -> list:     
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
    return text_splitter.split_documents(docs)

In [5]:
tgt_chunks = split_text(document)

In [6]:
openai_api_key = getpass()

 ········


In [7]:
neo4j_uri = getpass()

 ········


In [8]:
neo4j_username = getpass()

 ········


In [9]:
neo4j_password = getpass()

 ········


In [10]:
model_id = 'gpt-4o'

llm = ChatOpenAI(model=model_id, api_key=openai_api_key)
llm_transformer = LLMGraphTransformer(llm=llm)

In [13]:
register_graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

In [14]:
graph_documents = llm_transformer.convert_to_graph_documents(tgt_chunks)
register_graph.add_graph_documents(graph_documents)

In [15]:
qa_graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

In [16]:
chain = GraphCypherQAChain.from_llm(llm=llm, graph=qa_graph, allow_dangerous_requests=True, verbose=True)

In [17]:
qa_graph.get_structured_schema

{'node_props': {'Person': [{'property': 'id', 'type': 'STRING'}],
  'Place': [{'property': 'id', 'type': 'STRING'}],
  'Object': [{'property': 'id', 'type': 'STRING'}],
  'Creature': [{'property': 'id', 'type': 'STRING'}],
  'Animal': [{'property': 'id', 'type': 'STRING'}],
  'Location': [{'property': 'id', 'type': 'STRING'}],
  'Book': [{'property': 'id', 'type': 'STRING'}],
  'Organization': [{'property': 'id', 'type': 'STRING'}],
  'Event': [{'property': 'id', 'type': 'STRING'}]},
 'rel_props': {},
 'relationships': [{'start': 'Person', 'type': 'VISIT', 'end': 'Location'},
  {'start': 'Person', 'type': 'AWAITED', 'end': 'Person'},
  {'start': 'Person', 'type': '顔を見合わせる', 'end': 'Person'},
  {'start': 'Person', 'type': 'RECEIVE', 'end': 'Object'},
  {'start': 'Person', 'type': 'ACQUAINTED_WITH', 'end': 'Person'},
  {'start': 'Person', 'type': 'SAID', 'end': 'Person'},
  {'start': 'Person', 'type': '名付ける', 'end': 'Person'},
  {'start': 'Person', 'type': '育てました', 'end': 'Person'},
  {'

In [18]:
chain.invoke('桃太郎の仲間を教えてください')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {id: "桃太郎"})-[:ACCOMPANIED_BY]->(a:Animal)
RETURN a

Full Context:
[{'a': {'id': '犬'}}, {'a': {'id': '猿'}}]

> Finished chain.


{'query': '桃太郎の仲間を教えてください', 'result': '犬と猿が桃太郎の仲間です。'}